# Reboot - SQL Advanced

Tonight we will work with the `blog.sqlite` database is available at this URL:  
`https://wagon-public-datasets.s3.amazonaws.com/sql_databases/blog.sqlite`

In [ ]:
!curl https://wagon-public-datasets.s3.amazonaws.com/sql_databases/blog.sqlite > data/blog.sqlite

Let's have a look at our directory structure

In [ ]:
!tree

_TODO: Double click this cell and **paste** a screenshot of the schema for future reference_.

---
## 2. Most liked posts

Complete the code to get **the 3 most liked posts**:

In [4]:
import sqlite3

conn = sqlite3.connect("data/blog.sqlite")
c = conn.cursor()

# TODO: write the query
query = """
   select p.id,p.title ,count(l.id) as like_count 
FROM posts p 
JOIN likes l on l.post_id = p.id 
group by p.id 
order by like_count desc
limit 3 
"""

# TODO: Execute the query
c.execute(query)
rows = c.fetchall()
# TODO: Fetch and print the results
rows

[(143, 'Half imagine another.', 84),
 (83, 'Side foot leader popular.', 82),
 (99, 'Area paper whatever mean.', 81)]

---

### Pretty Print using _pandas_

The readbility of our `print()` statements is not so good.

Next week, we will introduce [pandas](https://pandas.pydata.org/) which will vastly improve the UX of our data exploration in Notebooks.

Execute the following cell to load `pandas` library:

In [5]:
import pandas as pd

Then try the previous `query` again, delegating the job of fetching results + displaying them to the `read_sql_query` function of `pandas`:

In [3]:
pd.read_sql_query(query, conn)

id                      title  like_count
0  143      Half imagine another.          84
1   83  Side foot leader popular.          82
2   99  Area paper whatever mean.          81

---
## 3. Find the three users who 'liked' the most

In [4]:
query = """
   SELECT 
	users.first_name || " " || users.last_name AS full_name, 
	COUNT(likes.id) AS nb_likes
FROM users
JOIN likes ON likes.user_id =users.id 
GROUP BY users.id
ORDER BY nb_likes DESC
LIMIT 3
"""

pd.read_sql_query(query, conn)

full_name  nb_likes
0    Michael Allen       236
1    Donna Ramirez       233
2  Hayley Williams       227

---
## 4. Find the most liked author

In [5]:
query = """
    SELECT COUNT(u.id) like_count, u.first_name, u.last_name, *
FROM posts p
JOIN likes l ON l.post_id = p.id  
JOIN users u ON u.id = p.user_id 
GROUP BY u.id
ORDER BY like_count DESC
LIMIT 1
"""

pd.read_sql_query(query, conn)

like_count first_name last_name  id  \
0         647     Teresa     Moore  40   

                                       title  \
0  Still relationship rock surface son wait.   

                                             content  user_id  id  created_at  \
0  Song place mother science talk action pattern ...       57  11  2019-10-24   

   user_id  post_id  id first_name last_name  
0       32       40  57     Teresa     Moore

---
## 5. Who are the authors of the 3 most liked posts?

In [6]:
query = """
   SELECT 
	COUNT(l.id) like_count,
	u.first_name || " " || u.last_name name
FROM posts p
JOIN likes l ON l.post_id = p.id  
JOIN users u ON u.id = p.user_id 
GROUP BY p.id
ORDER BY like_count DESC 
LIMIT 3
"""

pd.read_sql_query(query, conn)

like_count             name
0          84    Melissa Henry
1          82  Cynthia Raymond
2          81   Alexander Cook

---
## 6. How many people liked at least one post?

In [7]:
query = """
    WITH like_count AS (
	SELECT COUNT(l.id), user_id
	FROM likes l
	GROUP BY l.user_id
)
SELECT COUNT(*)
FROM like_count
"""

# SELECT COUNT(DISTINCT user_id)
# FROM likes


pd.read_sql_query(query, conn)

COUNT(*)
0        49

---
## 7. Compute the cumulative number of likes per day

In [7]:
query = """
    WITH likes_per_day AS (
	SELECT 
		l.created_at, 
		COUNT(l.id) as like_count
	FROM likes l
	GROUP BY created_at
)
SELECT
	*,
	SUM(like_count) OVER (
		ORDER BY created_at 
	) as cumul_likes_per_day
FROM likes_per_day
"""

pd.read_sql_query(query, conn)

created_at  like_count  cumul_likes_per_day
0    2019-01-01          24                   24
1    2019-01-02          34                   58
2    2019-01-03          40                   98
3    2019-01-04          36                  134
4    2019-01-05          27                  161
..          ...         ...                  ...
331  2019-12-24          35                 9882
332  2019-12-25          33                 9915
333  2019-12-26          32                 9947
334  2019-12-27          23                 9970
335  2019-12-28          30                10000

[336 rows x 3 columns]

---
## 8. (Optional) Who's the biggest fan/ fans of each author?

The biggest fan/ fans of an author is defined as the user or users who liked the author's posts the most. i.e. if there is a tie between fans that both liked an author 20 times, both fans should be returned alongside their like count and the author in question.
<br><br>
<details>
    <summary>💡 Click for Hint</summary>
    You might need to use <code>WITH</code>
</details>


In [10]:
query = """
    WITH l_counts AS(
        SELECT p.user_id as author, u.id as liker, count(u.id) as like_count
        FROM users u JOIN posts p JOIN likes l
            ON u.id = l.user_id AND l.post_id = p.id
        GROUP BY author, liker
    )
    SELECT 
        lliker.first_name || " " || lliker.last_name as liker,
        MAX(lc.like_count) as mlc, 
        aauthors.first_name || " " || aauthors.last_name as author
    FROM l_counts lc 
    JOIN users lliker JOIN users aauthors
        ON lc.liker = lliker.id AND lc.author = aauthors.id 
    GROUP BY lc.author 
    ORDER BY mlc
"""

pd.read_sql_query(query, conn)

liker  mlc           author
0     Donna Ramirez    3  Timothy Johnson
1      Amber Nguyen    4    Melissa Henry
2    Jessica Walker    6     Pamela Mason
3     Anthony Wiley    6      Susan Villa
4       Lisa Norris    7       Erin Smith
5    Scott Thompson    8     Tyler Wilson
6     Michael Allen   12   Brenda Griffin
7     Donna Ramirez   12  Madeline Porter
8      Michael Pugh   12       Ricky Cook
9       David Evans   12    Juan Lawrence
10    Barbara Hurst   16  Brittany Miller
11    Ashley Brooks   17      Lisa Palmer
12       Bruce Chen   17    Robert Dorsey
13   Scott Thompson   18     Mary Sanders
14    Barbara Hurst   18       David Boyd
15      James Payne   18     Eric Carroll
16    Ashley Brooks   19       Grace Kerr
17   Tristan Kelley   19   Brett Johnston
18     Sandra Davis   20   Melissa Nelson
19    Jeremiah Ruiz   20   Alexander Cook
20      Karen Brown   20      Donna Smith
21      Kaylee Ball   21  Jennifer Mendez
22    Maria Mccarty   24     Teresa Moore
23      Donald Gray   24  Cynthia Raymond
24  Hayley Williams   24      James Jones

## Yann's challenge
### The post that have more than ten likes